Imports

In [ ]:
import numpy as np
import pywt
import os
import cv2
#from google.colab.patches import cv2_imshow
from PIL import Image
from scipy.fftpack import dct
from scipy.fftpack import idct
from math import log10, sqrt

### Edit to change input images name

In [ ]:
image = 'lena.png'
watermark = 'logo.png'
recoveredLogoName = 'recovered_watermark.png'

## Convert Image to specific channel and size

In [ ]:
def convert_image(image_name, size):
  img = Image.open(image_name).resize((size, size), 1)
  img = img.convert('L')
  img.save(image_name)
  image_array = np.array(img.getdata(), dtype=np.float).reshape((size, size))
  return image_array

## Embed Watermark in given image

In [ ]:
 def embed_watermark(watermark_array, orig_image):
  watermark_array_size = watermark_array[0].__len__()
  watermark_flat = watermark_array.ravel()
  ind = 0
  for x in range (0, orig_image.__len__(), 8):
    for y in range (0, orig_image.__len__(), 8):
      if ind < watermark_flat.__len__():
        subdct = orig_image[x:x+8, y:y+8]
        subdct[5][5] = watermark_flat[ind]
        orig_image[x:x+8, y:y+8] = subdct
        ind += 1

  return orig_image

In [ ]:
 def apply_dct(image_array):
   size = image_array[0].__len__()
   all_subdct = np.empty((size, size))
   for i in range (0, size, 8):
     for j in range (0, size, 8):
       subpixels = image_array[i:i+8, j:j+8]
       subdct = dct(dct(subpixels.T, norm="ortho").T, norm="ortho")
       all_subdct[i:i+8, j:j+8] = subdct
   return all_subdct

## Inverse DCT

In [ ]:
 def inverse_dct(all_subdct):
  size = all_subdct[0].__len__()
  all_subidct = np.empty((size, size))
  for i in range (0, size, 8):
     for j in range (0, size, 8):
        subidct = idct(idct(all_subdct[i:i+8, j:j+8].T, norm="ortho").T, norm="ortho")
        all_subidct[i:i+8, j:j+8] = subidct
  return all_subidct

## Get Watermark

In [ ]:
 def get_watermark(dct_watermarked_coeff, watermark_size):
   subwatermarks = []
   for x in range (0, dct_watermarked_coeff.__len__(), 8):
     for y in range (0, dct_watermarked_coeff.__len__(), 8):
       coeff_slice = dct_watermarked_coeff[x:x+8, y:y+8]
       subwatermarks.append(coeff_slice[5][5])
   watermark = np.array(subwatermarks).reshape(watermark_size, watermark_size)
   return watermark

## Recover Watermark

In [ ]:
def recover_watermark(image_array):
  dct_watermarked_coeff = apply_dct(image_array)
  watermark_array = get_watermark(dct_watermarked_coeff, 128)
  watermark_array =  np.uint8(watermark_array)

  img = Image.fromarray(watermark_array)
  img.save(recoveredLogoName)

## Save Image from Array

In [ ]:
 def print_image_from_array(image_array, name):
  image_array_copy = image_array.clip(0, 255)
  image_array_copy = image_array_copy.astype("uint8")
  img = Image.fromarray(image_array_copy)
  img.save(name)

## Calculate PSNR

In [ ]:
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

# Watermarking Image

In [ ]:
def main(img):

   image_array = convert_image(image, 1024)
   watermark_array = convert_image(watermark, 128)

   dct_array = apply_dct(image_array)
   dct_array = embed_watermark(watermark_array, dct_array)
   coeffs_image = inverse_dct(dct_array)

   print_image_from_array(coeffs_image, 'image_with_watermark.png')
   recover_watermark(image_array =coeffs_image )

In [ ]:
main(image)

In [ ]:
original = cv2.imread(image)
compressed = cv2.imread("image_with_watermark.png")
a = PSNR(original, compressed)
print(a)

29.65897280204933


## Correlation Coefficient

In [ ]:
def correlation_coefficient(patch1, patch2):
    product = np.mean((patch1 - patch1.mean()) * (patch2 - patch2.mean()))
    stds = patch1.std() * patch2.std()
    if stds == 0:
        return 0
    else:
        product /= stds
        return product

In [ ]:
img1 = watermark
img2 = "recovered_watermark_without any_any_attack.png"
img1 = convert_image(img1, 128)
img2 = convert_image(img2, 128)
a = correlation_coefficient(img1, img2)
print("correlation_coefficient :",a)

correlation_coefficient : 0.9999926683491115


#Attacks

## Rotational Attack

In [ ]:
def rotational_attack(img):
  img = cv2.cvtColor(np.float32(img), cv2.COLOR_BGR2GRAY)
  center = (1024, 1024) # Get the image center
  rotation_matrix = cv2.getRotationMatrix2D(center, -45, 1) # Calculate the rotation matrix
  new_img = cv2.warpAffine(img, rotation_matrix, (1024, 1024)) # Transform input image
  cv2.imwrite('attack.png', new_img)

In [ ]:
img = cv2.imread("image_with_watermark.png")
rotational_attack(img)
img = convert_image("attack.png", 1024)
recover_watermark(image_array = img)

In [ ]:
img1 = watermark
img2 = "recovered_watermark_rotational_attack.png"
img1 = convert_image(img1, 128)
img2 = convert_image(img2, 128)
a = correlation_coefficient(img1, img2)
print("correlation_coefficient :",a)

correlation_coefficient : -0.049481430652508686


## Average Filter

In [ ]:
def averageFilter(img):

  m = img.shape[0]
  n = img.shape[1]
  filter = np.ones([3,3], dtype=int)
  filter = filter/9

  avgImg = np.zeros([m,n])
  for i in range(1, m-1):
    for j in range(1, n-1):
        temp = (img[i-1, j-1]*filter[0, 0])+(img[i-1, j]*filter[0, 1])+(img[i-1, j + 1]*filter[0, 2])+(img[i, j-1]*filter[1, 0])+ (img[i, j]*filter[1, 1])+(img[i, j + 1]*filter[1, 2])+(img[i + 1, j-1]*filter[2, 0])+(img[i + 1, j]*filter[2, 1])+(img[i + 1, j + 1]*filter[2, 2])
        avgImg[i,j] = temp
  cv2.imwrite('average.png', avgImg)
  return avgImg

In [ ]:
img = cv2.imread('image_with_watermark.png', 0)
averageFilter(img)
img = convert_image("average.png", 1024)
recover_watermark(image_array = img)

In [ ]:
img1 = watermark
img2 = "recovered_watermark_with_average_filter.png"
img1 = convert_image(img1, 128)
img2 = convert_image(img2, 128)
a = correlation_coefficient(img1, img2)
print("correlation_coefficient :",a)

correlation_coefficient : -0.014417694269850055


## Median Filter

In [ ]:
def medianFilter():
  img_noisy1 = cv2.imread('image_with_watermark.png', 0)
  m, n = img_noisy1.shape

  img_new1 = np.zeros([m, n])

  for i in range(1, m-1):
      for j in range(1, n-1):
          temp = [img_noisy1[i-1, j-1],
                img_noisy1[i-1, j],
                img_noisy1[i-1, j + 1],
                img_noisy1[i, j-1],
                img_noisy1[i, j],
                img_noisy1[i, j + 1],
                img_noisy1[i + 1, j-1],
                img_noisy1[i + 1, j],
                img_noisy1[i + 1, j + 1]]

          temp = sorted(temp)
          img_new1[i, j]= temp[4]

  img_new1 = img_new1.astype(np.uint8)
  cv2.imwrite('median.png', img_new1)

In [ ]:
medianFilter()
img = convert_image("median.png", 1024)
recover_watermark(image_array = img)

In [ ]:
img1 = watermark
img2 = "recovered_watermark_with_median_filter.png"
img1 = convert_image(img1, 128)
img2 = convert_image(img2, 128)
a = correlation_coefficient(img1, img2)
print("correlation_coefficient :",a)

correlation_coefficient : -0.1365381681723543


## Cropping Attack

In [ ]:
def croppingAttack():
  im = Image.open('image_with_watermark.png')
  print(img.shape[0])
  print(img.shape[1])
  im = im.crop( (1, 0, 1022, 1023) )
  im.save('cropped.png')


In [ ]:
croppingAttack()
img = convert_image("cropped.png", 1024)
recover_watermark(image_array = img)

1024
1024


In [ ]:
img1 = watermark
img2 = "recovered_watermark_with_cropping_attack.png"
img1 = convert_image(img1, 128)
img2 = convert_image(img2, 128)
a = correlation_coefficient(img1, img2)
print("correlation_coefficient :",a)

correlation_coefficient : 0.2195154736808018
